In [1]:
from cudf import DataFrame, Series
import cupy as cp

cp.set_printoptions(linewidth=150)


In [2]:
# n = 100, p = 4
X = DataFrame(
    {
        "a": cp.random.randint(low=-10, high=10, size=100),
        "b": cp.random.randint(low=-10, high=10, size=100),
        "c": cp.random.randint(low=-10, high=10, size=100),
        "C": [1] * 100,
    }
)

df = X.assign(d=X.a * 5)
X = df.to_cupy()


In [3]:
# conditional means
conditional = [df.groupby(col_name) for col_name in df.columns]
mean = [cond.mean() for cond in conditional]
var = [cond.var() for cond in conditional]


In [4]:
cp.linalg.matrix_rank(X)  # r < p


array(4)

In [5]:
# SVD
# X = UDV^T
U, S, VH = cp.linalg.svd(X)
U.shape, S.shape, VH.shape


((100, 100), (5,), (5, 5))

In [6]:
S


array([3.03498512e+02, 6.38137277e+01, 5.72160079e+01, 9.09889016e+00, 1.04045120e-14])

In [7]:
import numpy as cp
X1 = cp.array(
    (
        (-74, 80, 18),
        (14, -69, 21),
        (66, -72, -5),
        (-12, 66, -30),
        (3, 8, -7),
        (4, -12, 4),
    )
)

X2 = cp.array(
    ((-56, -112), (52, 104), (764, 1528), (4096, 8192), (-13276, -26552), (8421, 16842))
)

X1.T @ X2

(X := cp.hstack([X1, X2]))


array([[   -74,     80,     18,    -56,   -112],
       [    14,    -69,     21,     52,    104],
       [    66,    -72,     -5,    764,   1528],
       [   -12,     66,    -30,   4096,   8192],
       [     3,      8,     -7, -13276, -26552],
       [     4,    -12,      4,   8421,  16842]])

In [8]:
cp.linalg.matrix_rank(X1), cp.linalg.matrix_rank(X2)


(3, 1)

In [9]:
U, D, VT = cp.linalg.svd(X)
D


array([3.63684045e+04, 1.70701331e+02, 6.05331879e+01, 7.60190176e+00, 9.23124373e-13])

In [10]:
U, D, V = cp.linalg.svd(X)
U.shape, D.shape, V.shape


((6, 6), (5,), (5, 5))

In [11]:
cp.linalg.pinv(X.T @ X)


array([[5.46036229e-03, 3.82677443e-03, 6.93056814e-03, 0.00000000e+00, 0.00000000e+00],
       [3.82677443e-03, 2.73399993e-03, 4.90857229e-03, 0.00000000e+00, 0.00000000e+00],
       [6.93056814e-03, 4.90857229e-03, 9.41722012e-03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.51210343e-10, 3.02420686e-10],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.02420686e-10, 6.04841372e-10]])

condition indices defined as:

$$
\eta_k \equiv \frac{\mu_{max}}{\mu_k}
$$

where $\mu_k$ is the $k^{th}$ singular value


In [12]:
cp.linalg.norm(X) * cp.linalg.norm(cp.linalg.pinv(X))


4826.461448679665

In [13]:
# generalized inverse X+ of X is VD+ UT

# D+ is inverse of D
U, D, VT = cp.linalg.svd(X)


cp.dot(U[:, :5] * D, VT)


array([[-7.4000e+01,  8.0000e+01,  1.8000e+01, -5.6000e+01, -1.1200e+02],
       [ 1.4000e+01, -6.9000e+01,  2.1000e+01,  5.2000e+01,  1.0400e+02],
       [ 6.6000e+01, -7.2000e+01, -5.0000e+00,  7.6400e+02,  1.5280e+03],
       [-1.2000e+01,  6.6000e+01, -3.0000e+01,  4.0960e+03,  8.1920e+03],
       [ 3.0000e+00,  8.0000e+00, -7.0000e+00, -1.3276e+04, -2.6552e+04],
       [ 4.0000e+00, -1.2000e+01,  4.0000e+00,  8.4210e+03,  1.6842e+04]])

In [14]:
D.max() / D.min()  # YUP


3.939707969283433e+16

In [15]:
cp.linalg.pinv(X.T @ X)


array([[5.46036229e-03, 3.82677443e-03, 6.93056814e-03, 0.00000000e+00, 0.00000000e+00],
       [3.82677443e-03, 2.73399993e-03, 4.90857229e-03, 0.00000000e+00, 0.00000000e+00],
       [6.93056814e-03, 4.90857229e-03, 9.41722012e-03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.51210343e-10, 3.02420686e-10],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.02420686e-10, 6.04841372e-10]])

In [16]:
X1 = cp.array(
    (
        (-74, 80, 18),
        (14, -69, 21),
        (66, -72, -5),
        (-12, 66, -30),
        (3, 8, -7),
        (4, -12, 4),
    )
)

X2 = cp.array(
    ((-56, -112), (52, 104), (764, 1528), (4096, 8192), (-13276, -26552), (8421, 16842))
)

X1.T @ X2

(X := cp.hstack([X1, X2]))


array([[   -74,     80,     18,    -56,   -112],
       [    14,    -69,     21,     52,    104],
       [    66,    -72,     -5,    764,   1528],
       [   -12,     66,    -30,   4096,   8192],
       [     3,      8,     -7, -13276, -26552],
       [     4,    -12,      4,   8421,  16842]])

In [17]:
(X := cp.hstack([X1, X2]))
(n, p) = X.shape

# normalize
l = cp.power(cp.sum(cp.power(X, 2)), 0.5)
X = X / l

_, S, V = cp.linalg.svd(X)
λ = S  # cp.diag(S)

# ratio of largest SV to all
# condind = bsxfun(@rdivide,S(1,1),lambda);
condind = S[0] / S

# % variance decomposition proportions
# .* is element-wise multiplication
# phi_mat = bsxfun(@rdivide,V'.*V',lambda.^2);
phi_mat = cp.multiply(V, V) / cp.power(λ, 2)
phi = cp.sum(phi_mat)

# vdp = bsxfun(@rdivide,phi_mat,phi);
vdp = phi_mat / phi

# VIFs
#  vif = diag(inv(corr(X)));
vif = cp.diag(cp.linalg.inv(cp.corrcoef(X)))

fuzz = 0.0
s = "\n        Variance decomposition proportions\n\t"
# p = len(labels)
# for i in range(p):
#    temp = labels[i][:min(5, len(labels[i]))].ljust(7, ' ')
#    s += temp
s += "\nCondInd\n"
for i in range(p):
    s += "{:g}\t".format(round(condind[i]))
    for j in range(p):
        s += "{:2.3f}  ".format(vdp[i, j])
    s += "\n"
s += "\n   VIF: "
for i in range(p):
    s += "{:5.1f}  ".format(vif[i])
s += "\n"

print(s)



        Variance decomposition proportions
	
CondInd
1	0.000  0.000  0.000  0.000  0.800  
213	0.000  0.000  0.000  0.000  0.000  
601	0.000  0.000  0.000  0.000  0.000  
4784	0.000  0.000  0.000  0.000  0.000  
7.0993e+16	0.000  0.000  0.000  0.000  0.200  

   VIF: 3915733117946.5  4342021745497.8  364329869592405.2  13189775158915042.0   -0.0  



In [92]:
import numpy as np
import scipy.linalg
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pandas import read_csv, DataFrame, Series

def colldiag(X, labels=None, add_intercept=False, normalize=True, condind_thresh=20, vdp_thresh=0.5):
    if labels is None:
        labels = [f"X{i}" for i in range(X.shape[1])]
    elif not labels:
        labels = [f"X{i}" for i in range(X.shape[1])]

    if add_intercept:
        if np.all(X == 1):
            print("Intercept already present in design matrix. ADD_INTERCEPT parameter ignored.")
        else:
            X = np.hstack([np.ones((X.shape[0], 1)), X])
            labels.insert(0, 'int')
    n, p = X.shape

    if p != len(labels):
        raise ValueError("Labels don't match design matrix.")

    if normalize:
        # Normalize each column to unit length (pg 183 in Belsley et al)
        _len = np.sqrt(np.sum(X ** 2, axis=0)) 
        X = X/_len

    U, S, V = np.linalg.svd(X,full_matrices=False)

    
    # already diagonal values alone
    lambda_ = S #np.diag(S), might have to reshape into column
    condind = S[0] / lambda_
    phi_mat = (V.T * V.T) / (lambda_ ** 2) 
    phi = np.sum(phi_mat, axis=1).reshape(-1, 1) # expects COLUMN
    vdp = np.divide(phi_mat,phi).T # urgh

    # up to here is fine
    
    vdp_df = DataFrame(data=vdp, columns=labels)
    vdp_df = vdp_df.assign(condind=condind)
    vdp_df.insert(0, 'sv', range(1, 1+len(vdp_df)))
    vdp_df = vdp_df.set_index('sv')
    vif_df = Series(index=labels, data=vif)

    # need to find rows whre
    collinear = []
    for row in vdp_df.index:
        # filter for "high" condind
        s = vdp_df.loc[row][labels]
        if vdp_df.loc[row, 'condind'] > 30 and len(s[s > 0.5]) > 2:
            collinear_tuple = tuple(s[s > 0.5].index)
            collinear.append(collinear_tuple)

    return {
        'vdp': vdp_df, 
        'collinear': collinear
    }


info = colldiag(X, labels, normalize=True, add_intercept=True) 

In [197]:
from numpy import linalg

X = np.array([
    np.random.randint(-5, 5, size=(4)),
    np.random.randint(-5, 5, size=(4)),
    np.random.randint(-5, 5, size=(4)),
])

X = np.vstack([X, X[2, :]*1.5, X[1, :]/0.5, X[0, :]])

# adding constant
# X = np.hstack([np.ones((X.shape[0], 1)), X])

# n is number of rows in X
# p is number of variables
n, p = X.shape

(r := linalg.matrix_rank(X))
U, D, VH = linalg.svd(X)

# U = U.round(2)
# D = D.round(2)
# VH = VH.round(2)

D = np.diag(D) # output of numpy svd is row

# partition V into V1 \in (p\times r) and V2 \in (p\times (p-r))
V1 = V[:p, :r]
V2 = V[:p, -(p-r):]
V.shape, V1.shape, V2.shape

((5, 5), (4, 3), (4, 1))

In [198]:
(X @ V1)

array([[ 3.92817447, -2.17480867,  0.91632557],
       [ 2.12179558, -3.33423406,  1.54300576],
       [ 0.37725819, -5.7318771 ,  0.05710632],
       [ 0.56588729, -8.59781565,  0.08565948],
       [ 4.24359116, -6.66846812,  3.08601151],
       [ 3.92817447, -2.17480867,  0.91632557]])

In [199]:
# partition U into U\in (n\times r) and U\in (n\times (p-r))

U1 = U[:n, :r]
U2 = U[:n, -(p-r):]
U.shape, U1.shape, U2.shape

((6, 6), (6, 3), (6, 1))

In [200]:
D11 = D[:3, :3] # non-zeros of D

U1 @ D11

array([[ 1.9507716 , -5.11250166, -0.23836299],
       [ 4.98076596,  0.26782492,  1.45610448],
       [ 5.73588382,  0.71221161, -1.89536049],
       [ 8.60382573,  1.06831741, -2.84304074],
       [ 9.96153192,  0.53564984,  2.91220896],
       [ 1.9507716 , -5.11250166, -0.23836299]])

((6, 6), (6, 3), (6, 1))

In [179]:
X @ V2 # THIS SHOULD BE ZERO

array([[-2.68328157],
       [-3.57770876],
       [ 3.57770876],
       [ 5.36656315],
       [-7.15541753],
       [-2.68328157]])

In [77]:
# need to find rows whre
for row in vdp.index:
    # filter for "high" condind
    s = vdp.loc[row][labels]
    if vdp.loc[row, 'condind'] > 30 and len(s[s > 0.5]) > 2:
        collinear = s[s > 0.5].index
        print(collinear)
        print('\n')

Index(['C', 'DPI', 'dDPI'], dtype='object')




In [86]:
info['vdp']

,int,C,DPI,R,dDPI,condind
sv,,,,,,
1,0.001383,0.000003,0.000003,0.000244,0.001594,1.000000
2,0.003785,0.000010,0.000007,0.001425,0.135836,4.142638
3,0.310490,0.000028,0.000037,0.012988,0.000640,7.798541
4,0.263488,0.004662,0.004818,0.984368,0.048055,39.405786
5,0.420854,0.995297,0.995135,0.000975,0.813874,375.614256


## Why VIF Doesn't Work

       Variance decomposition proportions
	int    C      DPI    R      dDPI   
CondInd
1.0	0.001  0.000  0.000  0.000  0.002  
4.142637961897447	0.004  0.000  0.000  0.001  0.136  
7.798541471121814	0.310  0.000  0.000  0.013  0.001  
39.40578642668783	0.263  0.005  0.005  0.984  0.048  
375.61425582792555	0.421  0.995  0.995  0.001  0.814  
